In [1]:
import numpy as np
import itertools as it

Number of qubits

In [2]:
m = 2

# Load transversal
transversal_inv = load('2_transversal_inv.sobj')

Functions

In [3]:
def base(M, n):
    # calculate the image of the base under a matrix M
    
    # make a set of all combinations of the first column and the last n columns (these correspond to X_1, Z_1,...,Z_n)
    s = []
    for i in range(n+1, 2*n):
        s.append(M[0:2*n, i])
    powerset = it.chain.from_iterable(it.combinations(s, r) for r in range(1, len(s)+1))
    
    res = [vector(GF(2),2*n)]
        
    for i in powerset:
        v = vector(sum(i))     # calculate the sum of the elements of each combination (e.g IZZ = IZI + IIZ)
        res.append(v)
        
    return res

In [4]:
def pillars(M, n):
    # calculate the image of the pillars under a matrix M
    
    X1 = vector(M[0:2*n, 0])
    Z1 = vector(M[0:2*n, n])
    Y1 = X1 + Z1
    
    pI = base(M, n)
    pX = [(X1 + b) for b in pI]
    pY = [(Y1 + b) for b in pI]
    pZ = [(Z1 + b) for b in pI]
    
    return [pI, pX, pY, pZ]   

In [5]:
def tensor(A, n):
    # calculate the n fold tensor product of a matrix A
    
    kron = A
    count = 1
    while count < n:
        kron = np.kron(kron,A)
        count = count + 1
        
    if n == 2:
        res = np.reshape(kron, (4,4))
    elif n == 3:
        res = np.reshape(kron, (4,4,4)) 
    elif n == 4:
        res = np.reshape(kron, (4,4,4,4)) 
    elif n == 5:
        res = np.reshape(kron, (4,4,4,4,4)) 
        
    return res

In [9]:
def dist_stat(initial, M, n):
    # returns the success probability and the fidelity of an n-to-1 protocol M applied to an initial state
    pil = pillars(M, n)
    out = []
    for layer in pil:   
        coef = 0
        for elt in layer:
            if n == 2:
                coef = coef + initial[int(elt[0]) + 2*int(elt[n]), int(elt[1]) + 2*int(elt[n+1])]
            if n == 3:
                coef = coef + initial[int(elt[0]) + 2*int(elt[n]), int(elt[1]) + 2*int(elt[n+1]), \
                                   int(elt[2]) + 2*int(elt[n+2])]
            if n == 4:
                coef = coef + initial[int(elt[0]) + 2*int(elt[n]), int(elt[1]) + 2*int(elt[n+1]), \
                                   int(elt[2]) + 2*int(elt[n+2]), int(elt[3]) + 2*int(elt[n+3])]
            if n == 5:
                coef = coef + initial[int(elt[0]) + 2*int(elt[n]), int(elt[1]) + 2*int(elt[n+1]), \
                                   int(elt[2]) + 2*int(elt[n+2]), int(elt[3]) + 2*int(elt[n+3]), \
                                    int(elt[4]) + 2*int(elt[n+4])]
        out.append(coef)   
    sp = sum(out)
    fid = out[0]/sp

    return round(sp,10), round(fid,10)

In [7]:
def sucprob_fid_lists(initial, transversal_inv, n):
    # calculate the possible distillation statistics (success probability & fidelity) of the protocols in a transversal
    # applied to an initial state
          
    fid = []
    sp = []
    fslist = set()
    for key, M in transversal_inv.items():
        s, f = dist_stat(initial, M, n)
        if (s,f) not in fslist:
            sp.append(s)
            fid.append(f)
            fslist.add((s,f))

    return sp, fid

Results

In [8]:
# Bell diagonal state

init = tensor(vector([0.7, 0.15, 0.1, 0.05]), m)
print(sucprob_fid_lists(init, transversal_inv, m))

([0.75, 0.65, 0.71, 0.625, 0.8, 0.675, 0.68, 0.85, 0.745], [0.7, 0.7615384615, 0.7112676056, 0.788, 0.7, 0.737037037, 0.7352941176, 0.7, 0.6879194631])
